In [46]:
from typing import Tuple, List
import math
from scratch.probability import normal_cdf, inverse_normal_cdf
import random

In [7]:
def normal_approximation_to_binomial(n: int, p: float) -> Tuple[float, float]:
    mu = n * p
    sigma = math.sqrt(p * (1 - p) * n)
    return mu, sigma

In [8]:
normal_probability_below = normal_cdf


def normal_probability_above(lo: float, mu: float = 0, sigma: float = 1) -> float:
    return 1 - normal_cdf(lo, mu, sigma)


def normal_probability_between(
    lo: float, hi: float, mu: float = 0, sigma: float = 1
) -> float:
    return normal_cdf(hi, mu, sigma) - normal_cdf(lo, mu, sigma)


def normal_probability_outside(
    lo: float, hi: float, mu: float = 0, sigma: float = 1
) -> float:
    return 1 - normal_probability_between(lo, hi, mu, sigma)

In [13]:
def normal_upper_bound(probability: float, mu: float = 0, sigma: float = 1) -> float:
    return inverse_normal_cdf(probability, mu, sigma)


def normal_lower_bound(probability: float, mu: float = 0, sigma: float = 1) -> float:
    return inverse_normal_cdf(1 - probability, mu, sigma)


def normal_two_sided_bounds(
    probability: float, mu: float = 0, sigma: float = 1
) -> float:
    tail_probability = (1 - probability) / 2

    upper_bound = normal_lower_bound(tail_probability, mu, sigma)

    lower_bound = normal_upper_bound(tail_probability, mu, sigma)

    return lower_bound, upper_bound

In [10]:
mu_0, sigma_0 = normal_approximation_to_binomial(1000, 0.5)

In [14]:
lower_bound, upper_bound = normal_two_sided_bounds(0.95, mu_0, sigma_0)

In [ ]:
lo, hi = normal_two_sided_bounds(0.95, mu_0, sigma_0)

mu_1, sigma_1 = normal_approximation_to_binomial(1000, 0.55)

type_2_probability = normal_probability_between(lo, hi, mu_1, sigma_1)
power = 1 - type_2_probability

print(power)

0.8865480012953671


In [18]:
hi = normal_upper_bound(0.95, mu_0, sigma_0)

type_2_probability = normal_probability_below(hi, mu_1, sigma_1)
power = 1 - type_2_probability
print(power)

0.9363794803307173


In [ ]:
def two_sidded_p_value(x: float, mu: float = 0, sigma: float = 1) -> float:
    if x >= mu:
        return 2 * normal_probability_above(x, mu, sigma)
    else:
        return 2 * normal_probability_below(x, mu, sigma)

In [22]:
two_sidded_p_value(529.5, mu_0, sigma_0)

0.06207721579598835

In [57]:
extreme_value_count = 0

for _ in range(1000):
    num_heads = sum(1 if random.random() < 0.5 else 0 for _ in range(1000))

    if num_heads >= 530 or num_heads <= 470:
        extreme_value_count += 1

print(extreme_value_count)

65


In [ ]:
def run_experiment() -> List[bool]:

    return [random.random() < 0.5 for _ in range(1000)]


def reject_fairness(experiment: List[bool]) -> bool:

    num_heads = len([flip for flip in experiment if flip])

    return num_heads < 469 or num_heads > 531


random.seed(42)
experiments = [run_experiment() for _ in range(1000)]

num_rejections = len(
    [experiment for experiment in experiments if reject_fairness(experiment)]
)

print(num_rejections)

42


In [ ]:
def estimated_parameters(N: int, n: int) -> Tuple[float, float]:
    p = n / N
    sigma = math.sqrt(p * (1 - p) / N)
    return p, sigma


def a_b_test_statistic(N_A: int, n_A: int, N_B: int, n_B: int) -> float:
    p_A, sigma_A = estimated_parameters(N_A, n_A)
    p_B, sigma_B = estimated_parameters(N_B, n_B)

    return (p_B - p_A) / math.sqrt(sigma_A**2 + sigma_B**2)

In [65]:
z = a_b_test_statistic(1000, 200, 1000, 180)
print(z)

-1.1403464899034472


In [66]:
two_sidded_p_value(z)

0.254141976542236

In [67]:
z = a_b_test_statistic(1000, 200, 1000, 150)
two_sidded_p_value(z)

0.003189699706216853

In [ ]:
def B(alpha: float, beta: float) -> float:
    return math.gamma(alpha) * math.gamma(beta) / math.gamma(alpha + beta)


def beta_pdf(x: float, alpha: float, beta: float) -> float:

    if x <= 0 or x >= 1:
        return 0

    return x ** (alpha - 1) * (1 - x) ** (beta - 1) / B(alpha, beta)